<a href="https://colab.research.google.com/github/adilsonalbino/SQL-e-PYSPARK/blob/main/12_Realizando_Buscas_filtros_com_NOT_EXISTS%2C_e_LEFT_ANTI_JOINS_com_SQL_e_PYSPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#12-Realizando Buscas/filtros com NOT EXISTS, e LEFT ANTI JOINS com SQL e PYSPARK.ipynb


*   A ideia é filtrar dados de uma tabela somente se esses dados NÃO existir na tabela principal.



In [1]:
#instalando pyspark
!pip install pyspark

#importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#Criando app spark
spark = SparkSession.builder \
.appName("Spark")\
.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f98450eafeeeaa3dc02bb801cec7944aa7b53d21568c563a5f571769d2f8decc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
file_path_carros = "/content/drive/MyDrive/Colab Notebooks/PySpark_SQL/DATASET_CARROS/modelo_carro.csv"

In [3]:
df_carros = spark\
.read.format("csv")\
.option("header", True)\
.load(file_path_carros)\

df_carros.show(5)

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
|       5|      Escort|$77466.89|       17|
+--------+------------+---------+---------+
only showing top 5 rows



In [4]:
#Criando dois dataframes fitrados com where()

df_carros_inicio = df_carros.where(
    (col("id_carro") == '1') |
    (col("id_Carro") == '2') |
    (col("id_carro") == '3')
)

df_carros_fim = df_carros.where(
    (col("id_carro") == '1') |
    (col("id_Carro") == '2') |
    (col("id_carro") == '3') |
    (col("id_carro") == '4')

)

df_carros_inicio.show()
df_carros_fim.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
+--------+------------+---------+---------+

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
+--------+------------+---------+---------+



#Utilizando SQL

In [5]:
#Criando tabelas teporárias com os dfs
df_carros_inicio.createOrReplaceTempView("carros_inicio")
df_carros_fim.createOrReplaceTempView("carros_fim")

In [20]:
#Utilizando NOT EXISTS
#A função permite retornar apenas os dados existente que bater com o ID na outra tabela
df_carros_fim_sql = spark.sql("""
SELECT
  *
FROM carros_fim a
WHERE NOT EXISTS (SELECT 1 FROM carros_inicio b WHERE a.id_carro = b.id_carro)
""")

df_carros_fim_sql.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       4|          EX|$84981.12|       23|
+--------+------------+---------+---------+



#Utilizando PYSPARK

Obs: até o momento a função NOT EXISTS não é aplicável diretamente para operações de DataFrames no PySpark. Em vez disso, você deve usar a operação de junção (join) passando no finalo argumento "leftanti"


In [19]:
#Utilizando o LEFTSEMI para trazer o mesmo efeito do EXISTS

df_carros_spark = df_carros_fim.join(
    df_carros_inicio,
    df_carros_fim.id_carro == df_carros_inicio.id_carro,
    "leftanti"
    ).show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       4|          EX|$84981.12|       23|
+--------+------------+---------+---------+

